<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/milmor/NLP/blob/main/Notebooks/22_DeepSeek.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
</table>

# DeepSeek-R1

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import AutoTokenizer
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate
import time
import torch

In [2]:
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
number_gpus = 1
max_model_len = 8192
max_new_tokens = 256

In [3]:
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [4]:
terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.6,
    do_sample=True,
    return_full_text=True,
    eos_token_id=terminators,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=max_new_tokens)
hf = HuggingFacePipeline(pipeline=pipe)

In [5]:
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "What is an eigenvalue?"

start_time = time.time()
print(chain.invoke({"question": question}))
print(f"Generated in {time.time() - start_time} seconds.")

Question: What is an eigenvalue?

Answer: Let's think step by step. First, we need to understand what an eigenvalue is. An eigenvalue is a scalar associated with a linear transformation. It is a number that when a linear operator is applied to a vector, it scales the vector without changing its direction. So, to find the eigenvalues, we need to solve the equation Ax = λx, where A is a square matrix, x is a non-zero vector, and λ is the eigenvalue.

But wait, maybe I'm mixing up something. Is it Ax = λx or is it (A - λI)x = 0? I'm a bit confused. Let me try to recall. I think it's either of these two equations, but I need to remember which one leads to the eigenvalues.

So, maybe I should start by writing down the equation properly. If Ax = λx, then rearranging, we get (A - λI)x = 0. So, the eigenvalues are the scalars λ for which this equation has non-trivial solutions (i.e., x ≠ 0). That makes sense because if (A - λI) is singular, it means that there are non-trivial solutions, which 

In [6]:
def bytes_to_giga_bytes(bytes):
  return bytes / 1024 / 1024 / 1024

In [7]:
bytes_to_giga_bytes(torch.cuda.max_memory_allocated())

6.650988578796387